In [42]:
#importing the required packages for the algorithm
import csv
import os  
import pandas as pd

In [43]:
def readDataFromFile(filePath):
    #checking if the inputted file path is valid
    if os.path.isfile(filePath):
        with open(filePath, newline = '') as f:
            reader = csv.reader(f)
            tempData = list(reader)
            data = [[i for i in item if i != ''] for item in tempData]
            #reading the data from the input file into an array called data
            return data
    else:
        #printing an error message if the input file path is invalid
        print("Please input a valid file path.")

In [44]:
def dataPreparation(filePath):
    #checking if the inputted file path is valid
    if os.path.isfile(filePath):
        data = pd.read_csv(filePath)
        #reading the formatted data from the input file into an array called transactions
        transactions = [transaction[1]['itemDescription'].tolist() for transaction in list(data.groupby(['Member_number', 'Date']))]
        return transactions

In [45]:
class Node:
    def __init__(self, item, frequency, parent):
        self.item = item
        self.frequency = frequency
        self.next = None
        self.parent = parent
        self.children = {}
        
    def addFrequency(self, frequency):
        #adding to a specific node's frequency when called
        self.frequency += frequency
        
    def displayTree(self, index = 1):
        #printing a formatted version of the FP-tree when called
        print('|' * index, self.item, ' ', self.frequency)
        for child in self.children.values():
            child.displayTree(index + 1)

In [46]:
def constructTree(data, minSup = 1):
    #setting default value for minSup to 1
    headerTable = {}
    
    for transactions in data:
        for item in transactions:
            headerTable[item] = headerTable.get(item, 0) + data[transactions]
            
    for header in list(headerTable):
        if headerTable[header] < minSup:
            #removing items from the header table if it is smaller than minimum support
            del(headerTable[header])
            
    frequentItemSet = set(headerTable.keys())
    
    if len(frequentItemSet) == 0:
        #returning none if there are no items in the frequent item set
        return None, None
    
    for header in headerTable:
        #populating the header table with items
        headerTable[header] = [headerTable[header], None]
        
    #creating a FP-tree with a Null node as head
    toReturn = Node('Null', 1, None)
    
    for transactionSet, frequency in data.items():
        tempDict = {}
        for item in transactionSet:
            if item in frequentItemSet:
                #organizing the transaction items from header table to a temp dictionary
                tempDict[item] = headerTable[item][0]
        if len(tempDict) > 0:
            #sorting the items in the temp dictionary in reverse order and save it to sortedDict
            sortedDict = sorted(tempDict.items(), key = lambda p: p[1], reverse = True)
            orderedItems = [j[0] for j in sortedDict]
            #updating the tree with ordered frequent itemset
            updateTree(orderedItems, toReturn, headerTable, frequency)

    #returning the created FP-tree and the populated header table
    return toReturn, headerTable

In [47]:
def updateTree(items, inputTree, headerTable, frequency):
    if items[0] in inputTree.children:
        #increment frequency if items is present in the input tree
        inputTree.children[items[0]].addFrequency(frequency)   
    else:
        inputTree.children[items[0]] = Node(items[0], frequency, inputTree)
        if headerTable[items[0]][1] == None:
            #updating the header table with the children of the input tree
            headerTable[items[0]][1] = inputTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inputTree.children[items[0]])      
    if len(items) > 1:
        #calling the function updateTree if there are still remaining items
        updateTree(items[1::], inputTree.children[items[0]], headerTable, frequency)

In [48]:
def updateHeader(toUpdate, target):
    while toUpdate.next != None:
        #updating the header's next node
        toUpdate = toUpdate.next
    toUpdate.next = target

In [49]:
def formatData(data):
    toReturn = {}
    for transactions in data:
        #translating the input array into a frozen set to prevent error
        toReturn[frozenset(transactions)] = 1
    return toReturn

In [50]:
test = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Icecream', 'Eggs']]

#preparing the datasets to be inputted to the algorithm
test_data = formatData(test)
groceries_prepared = formatData(readDataFromFile('./datasets/groceries.csv'))
groceries_raw = formatData(dataPreparation('./datasets/groceries_raw.csv'))

In [51]:
#constructing the FP-tree by specifying input dataset and the minimum support
fpTree, headerTable = constructTree(groceries_prepared, 10)

In [52]:
#calling the displayTree function to visaulize the whole tree 
fpTree.displayTree()

| Null   1
|| Lassi   1841
||| Ghee   30
|||| Cheese   15
||||| Butter   8
|||||| Coffee Powder   4
||||||| Yougurt   2
|||||||| Tea Powder   1
||||||| Tea Powder   1
|||||| Yougurt   2
||||||| Tea Powder   1
|||||| Tea Powder   1
||||| Tea Powder   1
||||| Yougurt   2
|||||| Tea Powder   1
||||| Coffee Powder   3
|||||| Tea Powder   1
|||||| Yougurt   1
|||| Butter   7
||||| Coffee Powder   4
|||||| Yougurt   2
||||||| Tea Powder   1
|||||| Tea Powder   1
||||| Tea Powder   1
||||| Yougurt   1
|||| Coffee Powder   4
||||| Yougurt   2
|||||| Tea Powder   1
||||| Tea Powder   1
|||| Tea Powder   1
|||| Yougurt   2
||||| Tea Powder   1
||| Cheese   16
|||| Butter   8
||||| Tea Powder   1
||||| Yougurt   2
|||||| Tea Powder   1
||||| Coffee Powder   4
|||||| Yougurt   2
||||||| Tea Powder   1
|||||| Tea Powder   1
|||| Coffee Powder   4
||||| Yougurt   2
|||||| Tea Powder   1
||||| Tea Powder   1
|||| Yougurt   2
||||| Tea Powder   1
|||| Tea Powder   1
||| Milk   235
|||| Sugar   119
|||

In [53]:
def traverseTree(child, path):
    if child.parent:
        #traverse the FP-tree 
        path.append(child.item)
        traverseTree(child.parent, path)

In [54]:
def findPathByNode(target):
    path = {}
    while target:
        prefix = []
        traverseTree(target, prefix)
        if len(prefix) > 1:
            #finding the frequent path by a target node
            path[frozenset(prefix[1:])] = target.frequency
        target = target.next
    return path

In [55]:
def findPathByWorld(targetWord):
    #function to find the frequent path based on a target word
    if targetWord in headerTable.keys():
        return findPathByNode(headerTable[targetWord][1])
    else:
        print('Plese enter a valid word')

In [56]:
findPathByWorld('Butter')

{frozenset({'Cheese', 'Ghee', 'Lassi'}): 8,
 frozenset({'Cheese', 'Lassi'}): 8,
 frozenset({'Bread', 'Cheese', 'Panner'}): 8,
 frozenset({'Cheese', 'Sugar', 'Sweet'}): 7,
 frozenset({'Ghee', 'Panner'}): 8,
 frozenset({'Ghee', 'Lassi', 'Milk', 'Panner', 'Sugar'}): 7,
 frozenset({'Ghee', 'Lassi', 'Panner', 'Sweet'}): 8,
 frozenset({'Lassi', 'Milk', 'Panner', 'Sugar', 'Sweet'}): 8,
 frozenset({'Cheese', 'Ghee', 'Milk'}): 6,
 frozenset({'Cheese', 'Lassi', 'Milk', 'Panner'}): 7,
 frozenset({'Lassi', 'Milk', 'Sweet'}): 8,
 frozenset({'Cheese', 'Milk', 'Sugar'}): 7,
 frozenset({'Cheese', 'Ghee', 'Lassi', 'Milk', 'Panner'}): 6,
 frozenset({'Bread', 'Ghee', 'Milk', 'Panner'}): 7,
 frozenset({'Cheese', 'Panner', 'Sugar'}): 5,
 frozenset({'Bread', 'Cheese', 'Panner', 'Sugar'}): 8,
 frozenset({'Cheese', 'Ghee', 'Milk', 'Sugar'}): 8,
 frozenset({'Bread', 'Cheese', 'Lassi', 'Panner'}): 6,
 frozenset({'Bread', 'Ghee', 'Lassi', 'Milk', 'Sugar'}): 7,
 frozenset({'Bread', 'Cheese', 'Ghee'}): 7,
 frozens